In [1]:
# Using Pandas to analyse video game data to understand player count, demographics, purchase analytics and more
# Written by Matt Taylor
import pandas as pd

In [2]:
# Set the path of the file to analyse
game_data_path = "Resources/purchase_data.csv"

# Store the data into a dataframe
game_data_df = pd.read_csv(game_data_path)
game_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Looking at the data source to understand the contents
game_data_df.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [4]:
# Looking at the column names for future reference
game_data_df.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

### Player Count

* Total Number of Players

In [5]:
# Calculating the player count by finding the number of unique Usernames 
player_count = len(game_data_df["SN"].unique())
print(f"Total Number of Players: {player_count}")

Total Number of Players: 576


### Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [6]:
# Performing analysis on the purchase data and creating a summary table with a dataframe
unique_items = len(game_data_df["Item Name"].unique())
average_price = game_data_df["Price"].mean()
total_purchases = game_data_df["Purchase ID"].count()
total_revenue = game_data_df["Price"].sum()

purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [unique_items], "Average Purchase Price": average_price, "Total Number of Purchases": total_purchases, "Total Revenue": total_revenue})
purchasing_analysis_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [7]:
# Counting the total number of unique players and how the gender is distributed through the playerbase
total_players = len(game_data_df.groupby("SN"))
group_players = game_data_df.drop_duplicates(subset="SN").groupby("Gender").count()
group_players_df = pd.DataFrame(group_players["SN"]).rename(columns={"SN": "Number of Players"})
group_players_df["Percentage of Total Players"] = group_players_df["Number of Players"] / total_players

group_players_df

,Number of Players,Percentage of Total Players
Gender,,
Female,81,0.140625
Male,484,0.840278
Other / Non-Disclosed,11,0.019097


### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [8]:
# Calculating the difference in purchase data for each gender
purchase_count = game_data_df.groupby("Gender").count()
purchase_price = game_data_df.groupby("Gender").mean()
total_purchase_value = game_data_df.groupby("Gender").sum()

summary_gender_df = pd.DataFrame({"Purchase Count": purchase_count["SN"], "Average Purchase Price": purchase_price["Price"],  "Total Purchase Value": total_purchase_value["Price"]})
summary_gender_df["Average Purchase Total per Person by Gender"] = summary_gender_df["Total Purchase Value"] / group_players_df["Number of Players"]
summary_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


### Age Demographics

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

In [9]:
# Adding Age Group using bins of 4 years, noting that the minimum age in the dataset is 7 and maximum is 45
bins = (5, 10, 15, 20, 25, 30, 35, 40, 45)
group_names = ("5-9", "10-15","16-20","21-25","26-30","31-35","36-40", "41-45")

game_data_df["Age Group"] = pd.cut(game_data_df["Age"], bins, labels=group_names, include_lowest=True)

# Same calculations as previous cell on updated dataset
purchase_count = game_data_df.groupby("Age Group").count()
purchase_price = game_data_df.groupby("Age Group").mean()
total_purchase_value = game_data_df.groupby("Age Group").sum()

# Working out the number of unique players per age group
age_group_players = game_data_df.drop_duplicates(subset="SN").groupby("Age Group").count()
age_group_players_df = pd.DataFrame(age_group_players["SN"]).rename(columns={"SN": "Number of Players per Age Group"})

# Creating the summary table
summary_age_df = pd.DataFrame({"Purchase Count": purchase_count["SN"], "Average Purchase Price": purchase_price["Price"],  "Total Purchase Value": total_purchase_value["Price"]})
summary_age_df["Average Purchase Total per Person by Age Group"] = summary_age_df["Total Purchase Value"] / age_group_players_df["Number of Players per Age Group"]

summary_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Age Group
Age Group,,,,
5-9,32,3.405000,108.96,4.540000
10-15,54,2.900000,156.60,3.819512
16-20,200,3.107800,621.56,4.143733
21-25,325,3.020431,981.64,4.231207
26-30,77,2.875584,221.42,3.752881
31-35,52,2.994423,155.71,4.208378
36-40,33,3.404545,112.35,4.321154
41-45,7,3.075714,21.53,3.075714


### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [10]:
# Finding the top 5 spenders in the game by sum of purchase value. Also removing the unnecessary columns using filtering
top_spenders = game_data_df.groupby("SN").sum().sort_values(by=["Price"], ascending=False).head(5)
top_spenders = top_spenders["Price"]

# Finding the number of transactions per username and removing uneccessary columns using filtering
top_spender_purchase_count = game_data_df.groupby("SN").count()
top_spender_purchase_count = top_spender_purchase_count["Purchase ID"]

# Using the merge function to add the number of purchases to the top 5 spenders, and adding final column for average price
top_spender_summary_df = pd.merge(top_spenders, top_spender_purchase_count, on="SN").rename(columns={"Purchase ID": "Number of Purchases"})
top_spender_summary_df["Average Purchase Price"] = top_spender_summary_df["Price"] / top_spender_summary_df["Number of Purchases"]
top_spender_summary_df

,Price,Number of Purchases,Average Purchase Price
SN,,,
Lisosia93,18.96,5,3.792000
Idastidru52,15.45,4,3.862500
Chamjask73,13.83,3,4.610000
Iral74,13.62,4,3.405000
Iskadarya95,13.10,3,4.366667


### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [11]:
# Find the items with the most transactions using a count groupby
most_popular = game_data_df.groupby("Item Name").count().sort_values(by=["Item ID"], ascending=False).head(5)
most_popular = most_popular["Item ID"]

# Item ID. Get a list on unique item IDs
most_popular_id = game_data_df.drop_duplicates(subset="Item Name").groupby("Item Name").sum()
most_popular_id = most_popular_id["Item ID"]

# Item price. The Item price changes between transactions so we will use the average price
most_popular_price = game_data_df.groupby("Item Name").mean()
most_popular_price = most_popular_price["Price"]

# Start creating the summary table by merging the previously created series and dataframes
most_popular_summary = pd.merge(most_popular, most_popular_price, on="Item Name").rename(columns={"Item ID": "Number of Purchases", "Price": "Average Price"})
most_popular_summary = pd.merge(most_popular_summary, most_popular_id, on="Item Name")

# Calculating the last column for the total purchase value
most_popular_summary["Total Purchase Value"] = most_popular_summary["Number of Purchases"] * most_popular_summary["Average Price"]
most_popular_summary

,Number of Purchases,Average Price,Item ID,Total Purchase Value
Item Name,,,,
Final Critic,13,4.614615,92,59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,178,50.76
Persuasion,9,3.221111,132,28.99
Nirvana,9,4.900000,82,44.10
"Extraction, Quickblade Of Trembling Hands",9,3.530000,108,31.77


### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [12]:
# Fid the items that have the highest sum of each transaction price using sum groupby
most_profitable = game_data_df.groupby("Item Name").sum().sort_values(by=["Price"], ascending=False).head(5)
most_profitable = most_profitable["Price"]

# Item ID. Get a list on unique item IDs
most_profitable_id = game_data_df.drop_duplicates(subset="Item Name").groupby("Item Name").sum()
most_profitable_id = most_profitable_id["Item ID"]

# Item price. The Item price changes between transactions so we will use the average price
most_profitable_price = game_data_df.groupby("Item Name").mean()
most_profitable_price = most_profitable_price["Price"]

# Purchase Count. The purchase count can be calculated by using the count of any column and groupby
most_profitable_count = game_data_df.groupby("Item Name").count()
most_profitable_count = most_profitable_count["SN"]

# Creating the summary table by merging the previously created series and dataframes
most_profitable_summary = pd.merge(most_profitable, most_profitable_id, on="Item Name")# .rename(columns={"Item ID": "Number of Purchases", "Price": "Average Price"})
most_profitable_summary = pd.merge(most_profitable_summary, most_profitable_price, on="Item Name")
most_profitable_summary = pd.merge(most_profitable_summary, most_profitable_count, on="Item Name").rename(columns={"Price_x": "Total Purchase Value", "Price_y": "Item Price", "SN": "Purchase Count"})

most_profitable_summary

,Total Purchase Value,Item ID,Item Price,Purchase Count
Item Name,,,,
Final Critic,59.99,92,4.614615,13
"Oathbreaker, Last Hope of the Breaking Storm",50.76,178,4.230000,12
Nirvana,44.10,82,4.900000,9
Fiery Glass Crusader,41.22,145,4.580000,9
Singed Scalpel,34.80,103,4.350000,8
